In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install faiss-cpu numpy pillow soundfile tqdm matplotlib scipy
!pip install git+https://github.com/facebookresearch/ImageBind.git
!pip install git+https://github.com/LAION-AI/CLAP.git
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/facebookresearch/ImageBind.git to /tmp/pip-req-build-pm0vf6sr
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind.git /tmp/pip-req-build-pm0vf6sr
  Resolved https://github.com/facebookresearch/ImageBind.git to commit cfee5753e8e0b7a9df8aeb19c493e8facd81a5f2
  Preparing metadata (setup.py) ... done
  Using cached pytorchvideo-0.1.5-py3-none-any.whl
  Cloning https://github.com/LAION-AI/CLAP.git to /tmp/pip-req-build-92lkocdh
  Running command git clone --filter=blob:none --quiet https://github.com/LAION-AI/CLAP.git /tmp/pip-req-build-92lkocdh
  Resolved https://github.com/LAION-AI/CLAP.git to commit 1fd4c37df5ffbfcfbad5415c170bc66cf94c9994
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nkyoxi_m
  Run

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [ ]:
!mkdir Data/Audio

mkdir: cannot create directory ‘Data/Audio’: File exists


In [ ]:
import torch, numpy as np

# Allow older numpy dtype objects and scalars
torch.serialization.add_safe_globals([
    np.core.multiarray.scalar,
    np.dtype,
    np.dtypes.Float64DType
])

In [ ]:
from laion_clap import CLAP_Module

clap_model = CLAP_Module(enable_fusion=False)
clap_model.load_ckpt()   # will now succeed
clap_model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

print("✅ CLAP model loaded successfully.")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ CLAP model loaded successfully.


In [ ]:
import os
import torchaudio
import faiss
import json
from imagebind.models import imagebind_model
import imagebind.data as ib_data

# ------------------------------------------------
# 1️⃣ Setup
# ------------------------------------------------
AUDIO_PATH = "/content/Data/Audio"
SAVE_DIR = "/content/drive/MyDrive/Colab Notebooks/Context_Aware_Audio"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Optionally load ImageBind for future fusion (not used here)
ib_model = imagebind_model.imagebind_huge(pretrained=True).eval().to(device)
print("✅ ImageBind model loaded (for fusion layer).")

# ------------------------------------------------
# 2️⃣ Extract audio embeddings with CLAP
# ------------------------------------------------
audio_embeds = []
audio_meta = []

for file in sorted(os.listdir(AUDIO_PATH)):
    if not file.lower().endswith(".wav"):
        continue

    file_path = os.path.join(AUDIO_PATH, file)
    try:
        wav, sr = torchaudio.load(file_path)
        with torch.no_grad():
            # Get audio embeddings from CLAP
            audio_embed = clap_model.get_audio_embedding_from_data(
                x=wav.to(device),
                use_tensor=True
            )
        audio_embed_np = audio_embed.cpu().numpy().astype("float32")
        audio_embeds.append(audio_embed_np)
        audio_meta.append({"filename": file})
        print(f"Processed: {file}")

    except Exception as e:
        print(f"⚠️ Skipped {file}: {e}")

# Stack embeddings and normalize
audio_matrix = np.vstack(audio_embeds)
faiss.normalize_L2(audio_matrix)

# ------------------------------------------------
# 3️⃣ Build FAISS index
# ------------------------------------------------
dim = audio_matrix.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine similarity
index.add(audio_matrix)

# ------------------------------------------------
# 4️⃣ Save FAISS index and metadata
# ------------------------------------------------
os.makedirs(SAVE_DIR, exist_ok=True)
faiss.write_index(index, f"{SAVE_DIR}/audio.index")
json.dump(audio_meta, open(f"{SAVE_DIR}/audio_meta.json", "w"))

print(f"\n✅ Audio index saved in {SAVE_DIR}")
print(f"✅ Total audio files indexed: {len(audio_meta)}")


✅ ImageBind model loaded (for fusion layer).


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Processed: file_example_WAV_1MG.wav

✅ Audio index saved in /content/drive/MyDrive/Colab Notebooks/Context_Aware_Audio
✅ Total audio files indexed: 1
